In [1]:
import re
import time
import requests
import numpy as np
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from matplotlib import pyplot as plt, rcParams
from selenium import webdriver
import pandas as pd
from pydataset import data

# Gathering and combining data into data frame

In [2]:
all_data = {}

# get data from data files
data_files = ["demographic","diet","examination", "labs", "questionnaire"]
for filename in data_files:
    data = pd.read_csv(filename + ".csv")
    all_data[filename] = data

# combine all data into single data frame
data = pd.merge(all_data[data_files[0]],all_data[data_files[1]],on="SEQN")
for j in range(2,len(data_files)):
    data = pd.merge(data,all_data[data_files[j]])

# Scraping variable name information from website

### Updated from previously:

More refined list of variables to keep for analysis.

### Returns: 

Dictionary with variable descriptions and list of variables to keep in data frame. 

In [3]:
# crawl through website to get variable names and descriptions
def get_variable_info():
    """
    Crawl through the NHANES site to get variable 
    names and descriptions for data.
    """

    variable_info = {}
    
    to_keep = []
    
    # these are the variables that we are interested in analysing
    to_keep_list = ['URXUMA','DED125','ALQ120Q','ALQ120U',
                    'URXUMS','URXCRS','URDACT','LBXTC','LBXWBCSI',
                    'LBXRBCSI','LBXHGB','LBXPLTSI','LBXSCH','LBXSCA',
                    'LBXSGL','LBXSIR','RIAGENDR','RIDAGEYR',
                    'DMDCITZN','DMDEDUC2','DMDMARTL',
                    'RIDEXPRG','INDFMIN2','INDFMPIR','DMDFMSIZ',
                    'RIDRETH3','DMDHHSZA','DMDHHSZB',
                    'DMDHHSZE','DUQ240','MCQ220','BMXWAIST','BMXBMI',
                    'BMXHT','BMXWT','BPXPLS','BPXSY3','BPXSY2',
                    'BPXDI3','BPXDI2','DBD100','DRQSPREP','DRQSDIET',
                    'RHQ420','RHQ540','SXD021','SLD010H','OCQ180',
                    'SMQ020','WHD020','WHD010','WHD050','WHD030','WHD070',
                    'DAQ665','PAD680','PAQ710','PAQ715']

    # these are the variables relating to mental health
    to_keep_re = re.compile('^DPQ[0-9]{3}$')
    
    datatype_finder = re.compile(r"^(Demographics|Dietary|Examination|Laboratory|Questionnaire)$")
    varpage_finder = re.compile(r"^NHANES 2013-2014.*Variable List$")
    the_base = "https://wwwn.cdc.gov"
    base_url = "https://wwwn.cdc.gov/nchs/nhanes/continuousnhanes/default.aspx?BeginYear=2013"      
    base_soup = BeautifulSoup(requests.get(base_url).text, "html.parser")
    page_tags = base_soup.find_all(name='a', href=True, string=datatype_finder)
    pages = [the_base + tag.attrs["href"] for tag in page_tags]
    
    for page in pages:
        variable_names = []
        variable_descriptions = []
        end = page.find('&')
        key = page[64:end]
        time.sleep(1)           
        soup = BeautifulSoup(requests.get(page).text, "html.parser")
        new_base = the_base + "/nchs/nhanes/search/"
        next_page = new_base + soup.find_all(name="a", string=varpage_finder, href=True)[0].attrs["href"]
        
        time.sleep(1)
        new_soup = BeautifulSoup(requests.get(next_page).text, "html.parser")
        names = new_soup.find_all(name="tr")[2:]

        for name in names:
            temp = name.find_all(name="td")
            varname = temp[0].text
            desc = temp[1].text
            
            if varname in to_keep_list or re.search(to_keep_re, varname):
                to_keep.append(varname)
                variable_info[varname] = desc
            
    return variable_info, to_keep

var_info, to_keep = get_variable_info()


# Feature Engineering

#### Dropping unwanted variables from table

The data set contains over 1200 pieces of information for each data point, not all of these are relevant. This step removes information about data points that is not relevant to mental health. 

#### Replacing multiple blood pressure data columns with single average

The examination included taking blood pressure multiple times to get a trusted value for each person. This step averages these measurements to simplify analysis. 

#### Adjusting alcohol consumption to account for different units

The question regarding alcohol consumption left units ambiguous and in the following question the units are specified. This step creates a new column in which the units are specified and uniform. 

#### Creating separate data frames with variables only pertaining to women (dropping these variables from main data frame)

There are a few question regarding pregnancy and birth control methods. These questions only pertain to mature women so the columns are mainly NaN. This step creates a separate data frame called 'women' to store this information along with mental health data. The women information is then dropped from the main data frame. 

#### Eliminating rows of table that have NaN for at least 20 of the columns

There are over 2000 data points that have NaN for values in at least 20 of the 59 columns in the data frame. This step eliminates these rows to make analysis more uniform. 

In [4]:
# generate drop list from list of variables to keep and drop unwanted
drop_list = [c for c in data.columns if c not in to_keep]
data.drop(drop_list,axis=1, inplace=True)

# average blood pressure data and replace with averages
data['BPXSY'] = data[['BPXSY3','BPXSY2']].mean(axis=1)
data['BPXDI'] = data[['BPXDI3','BPXDI2']].mean(axis=1)
data.drop(['BPXSY3','BPXSY2','BPXDI3','BPXDI2'], axis=1, inplace=True)
var_info['BPXSY'] = 'Systolic:  Blood pressure mm Hg'
var_info['BPXDI'] = 'Diastolic: Blood pressure mm Hg'

# adjust alcohol consumption data for different units
data[['ALQ120Q','ALQ120U']]
units = {1:(1/7),2:(12/365),3:(1/365)}
data = data.replace({'ALQ120U':units})
data['ALQ120'] = data['ALQ120Q'].multiply(data['ALQ120U'], axis='index')
data.drop(['ALQ120Q','ALQ120U'], axis=1, inplace=True)
var_info['ALQ120'] = 'Number of times alcohol is consumed per day'

# create separate data frame containing data relating to pregnancy
women_ = re.compile(r'^(RIDEXPRG|RHQ)')
mental_ = re.compile(r'^DPQ')
colw = [c for c in data.columns if re.match(women_, c) or re.match(mental_, c)]
women = data[colw].dropna()
colt = [c for c in data.columns if re.match(women_, c)]
data.drop(colt, axis=1, inplace=True)

# drop rows that have NaN for at least 20 columns
null_count = data.isnull().sum(axis=1)
data = data[null_count <= 20]

data

,RIAGENDR,RIDAGEYR,RIDRETH3,DMDCITZN,DMDEDUC2,DMDMARTL,DMDFMSIZ,DMDHHSZA,DMDHHSZB,DMDHHSZE,...,PAQ715,SLD010H,SMQ020,SXD021,WHD010,WHD020,WHD050,BPXSY,BPXDI,ALQ120
0,1,69,4,1.0,3.0,4.0,3,0,0,2,...,8.0,7.0,1.0,1.0,69.0,180.0,210.0,108.0,75.0,0.002740
1,1,54,3,1.0,3.0,1.0,4,0,2,0,...,8.0,9.0,1.0,1.0,71.0,200.0,160.0,158.0,61.0,1.000000
2,1,72,3,1.0,4.0,1.0,2,0,0,2,...,0.0,8.0,1.0,NaN,70.0,195.0,195.0,143.0,78.0,NaN
4,2,73,3,1.0,5.0,1.0,2,0,0,2,...,1.0,9.0,2.0,NaN,67.0,120.0,150.0,138.0,87.0,NaN
5,1,56,1,1.0,4.0,3.0,1,0,0,0,...,8.0,5.0,1.0,1.0,64.0,235.0,240.0,156.0,81.0,0.013699
7,2,61,3,1.0,5.0,2.0,1,0,0,1,...,0.0,9.0,2.0,1.0,64.0,212.0,212.0,125.0,81.0,0.005479
8,2,56,3,1.0,3.0,3.0,7,3,0,0,...,8.0,6.0,1.0,1.0,61.0,137.0,137.0,119.0,72.0,0.142857
9,1,65,3,1.0,2.0,2.0,1,0,0,1,...,0.0,7.0,1.0,1.0,71.0,165.0,165.0,142.0,77.0,0.571429
10,2,26,3,1.0,5.0,5.0,3,0,0,0,...,8.0,8.0,2.0,2.0,60.0,105.0,110.0,104.0,64.0,0.285714
12,1,76,3,1.0,5.0,1.0,2,0,0,2,...,1.0,6.0,2.0,NaN,69.0,224.0,222.0,128.0,68.0,0.005479


# Descriptions of each column name (with units when appicable)

In [5]:
for v in sorted(data.columns):
    print(v, var_info[v])

ALQ120 Number of times alcohol is consumed per day
BMXBMI Body Mass Index (kg/m**2)
BMXHT Standing Height (cm)
BMXWAIST Waist Circumference (cm)
BMXWT Weight (kg)
BPXDI Diastolic: Blood pressure mm Hg
BPXPLS 60 sec. pulse (30 sec. pulse * 2)
BPXSY Systolic:  Blood pressure mm Hg
DBD100 How often {do you/does SP} add ordinary salt to {your/his/her/SP's} food at the table?  Would you say . . .
DED125 During the past 30 days, how much time did you usually spend outdoors between 9 in the morning and 5 in the afternoon on the days when you were not working or going to school?
DMDCITZN {Are you/Is SP} a citizen of the United States? [Information about citizenship is being collected by the U.S. Public Health Service to perform health related research. Providing this information is voluntary and is collected under the authority of the Public Health Service Act. There will be no effect on pending immigration or citizenship petitions.]
DMDEDUC2 What is the highest grade or level of school {you h

# Keys for data in certain columns

### Unless otherwise specified

1: Yes

2: No

7: Refused

9: Don't know



### DBD100

1: Rarely

2: Occasionally

3: Very often	



### DMDEDUC2

1: Less than 9th grade

2: 9-11th grade or no diploma

3: High school diploma/GED

4: Some college or AA

5: College degree and or more


### DPQ010 - DPQ090

0: Not at all

1: Several days

2: Over half of days

3: Nearly every day


### DPQ100:

0: Not at all difficult

1: Somewhat difficult

2: Very difficult

3: Extremely difficult


### DRQSPREP

1: Never	

2: Rarely

3: Occasionally

4: Very Often



### PAQ710, PAQ715

8: None



### RIAGENDER

1: Male

2: Female



### RIAMARTL

1: Married

2: Widowed

3: Divorced

4: Separated

5: Never married

6: Living with partner


### RIDRETH3

1: Mexican American

2: Hispanic

3: White

4: Black

5: Asian

6: Other



### SMQ0404

1: Everyday

2: Some days

3: Not at all



### WHQ030

1: Overweight

2: Underweight

3: Seems about right
